In [2]:
from datasets import load_dataset
import polars as pl
import os

In [3]:
# Get the directory above the current directory
above_dir = os.path.dirname(os.getcwd())

# Specify the cache directory as 'data' subdirectory within the current directory
cache_dir = os.path.join(above_dir, "cache")
print(cache_dir)

c:\Users\Tommaso\Documents\Dev\Cynde\cache


In [3]:
dataset = load_dataset("teknium/OpenHermes-2.5", cache_dir)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to C:/Users/Tommaso/.cache/huggingface/datasets/teknium___json/teknium--OpenHermes-2.5-39fceffcecb81b46/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['hash', 'model_name', 'model', 'skip_prompt_formatting', 'source', 'custom_instruction', 'language', 'system_prompt', 'avatarUrl', 'topic', 'conversations', 'views', 'id', 'category', 'idx', 'title'],
        num_rows: 1001551
    })
})

In [4]:
dataset_name = "OpenHermes-2.5"
dataset_path = os.path.join(cache_dir, dataset_name)


In [ ]:
dataset["train"].to_parquet(dataset_path+".parquet")

In [5]:
df = pl.read_parquet(dataset_path+".parquet")


In [14]:
df["source"].value_counts().sort("count")

source,count
str,u32
"""Econ_domain_ex…",660
"""lmsys1m""",1631
"""caseus_custom""",2688
"""LMSys Chatbot …",3136
null,4192
"""CogStackMed""",4443
"""UnnaturalInstr…",8610
"""GPT-4 Comparis…",14928
"""platypus""",22280


In [11]:
for i in df["source"].unique():
    print(i)

LMSys Chatbot Arena
airoboros2.2

CamelAI
platypus
caseus_custom
lmsys1m
glaive-code-assist
None
EvolInstruct_70k
UnnaturalInstructions
GPT-4 Comparison Data
cot_alpaca_gpt4
metamath
CogStackMed
Econ_domain_expert


In [18]:
targets = ["LMSys Chatbot Arena","caseus_custom"]
filtered_df = df.filter(pl.col("source").is_in(targets)).select("conversations","source")
filtered_path = dataset_path+"_filtered.parquet"
filtered_df.write_parquet(filtered_path)